# Initial Import

In [1]:
from tqdm import tqdm
import os
import numpy as np
import torch
from torch import optim
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd

from main import backgammon

/Applications/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


pygame 2.5.2 (SDL 2.28.3, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


## Import Dataset

In [2]:
def load_dataset(file_path):
    # Load CSV using pandas
    df = pd.read_csv(file_path)
    
    # Split into features and labels
    data = df.values
    X = torch.tensor(data[:, :-1], dtype=torch.float32)
    y = torch.tensor(data[:, -1], dtype=torch.float32).unsqueeze(1)  # Ensure shape (N, 1)
    
    return TensorDataset(X, y)

train_file = os.path.join("..","Data","Deep","BoardEquity",'train.txt')
validation_file = os.path.join("..","Data","Deep","BoardEquity",'validation.txt')
test_file = os.path.join("..","Data","Deep","BoardEquity",'test.txt')

train_dataset = load_dataset(train_file)
validation_dataset = load_dataset(validation_file)
test_dataset = load_dataset(test_file)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(validation_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)

# Example: iterate over a batch from the train set
for X_batch, y_batch in train_loader:
    print("Input shape:", X_batch.shape)
    print("Labels shape:", y_batch.shape)
    break

Input shape: torch.Size([64, 578])
Labels shape: torch.Size([64, 1])


In [3]:
class BGNet(nn.Module):
    def __init__(self):
        super(BGNet, self).__init__()
        # Shared input-to-hidden layer: both board states use the same weights.
        self.fc_shared = nn.Linear(289, 12)
        # Hidden-to-output layer: we use a single layer that will be applied
        # to both hidden representations. In the forward pass we explicitly
        # flip the sign for one branch to enforce that its effect is the negative.
        self.fc_out = nn.Linear(12, 1)

    def forward(self, board_left, board_right):
        # Process left board: apply shared fc layer and ReLU activation.
        h_left = F.relu(self.fc_shared(board_left))
        # Process right board: same shared fc layer.
        h_right = F.relu(self.fc_shared(board_right))
        
        # Compute the output for each branch using the same fc_out layer.
        out_left = self.fc_out(h_left)
        # Explicitly invert the output from the right branch.
        out_right = -self.fc_out(h_right)
        
        # The network's decision is the sum of these contributions.
        # This is equivalent to computing the difference between the two evaluations.
        diff = out_left + out_right
        
        # Apply a sigmoid activation so that final output is between 0 and 1.
        final_output = torch.sigmoid(diff)
        return final_output

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Assume: model is your instance of BGNet
model = BGNet()

# Loss function: Binary Cross-Entropy (good for sigmoid output)
criterion = nn.BCELoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training settings
num_epochs = 200  # Change as needed
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def evaluate(model, data_loader):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in data_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch[:, :289], X_batch[:, 289:])
            loss = criterion(outputs, y_batch)
            total_loss += loss.item() * y_batch.size(0)

            preds = (outputs > 0.5).float()
            correct += (preds == y_batch).sum().item()
            total += y_batch.size(0)
    
    avg_loss = total_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

# === Training Loop ===
for epoch in range(1, num_epochs + 1):
    model.train()
    running_loss = 0
    correct = 0
    total = 0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        board_left = X_batch[:, :289]
        board_right = X_batch[:, 289:]

        outputs = model(board_left, board_right)
        loss = criterion(outputs, y_batch)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * y_batch.size(0)

        # For training accuracy
        preds = (outputs > 0.5).float()
        correct += (preds == y_batch).sum().item()
        total += y_batch.size(0)

    avg_train_loss = running_loss / total
    train_acc = correct / total

    if epoch % 10 == 0:
        val_loss, val_acc = evaluate(model, val_loader)
        test_loss, test_acc = evaluate(model, test_loader)
        
        print(f"Epoch {epoch:03d}:")
        print(f"  Train Loss: {avg_train_loss:.4f}, Accuracy: {train_acc*100:.2f}%")
        print(f"  Val   Loss: {val_loss:.4f}, Accuracy: {val_acc*100:.2f}%")
        print(f"  Test  Loss: {test_loss:.4f}, Accuracy: {test_acc*100:.2f}%\n")

    torch.save(model.state_dict(), "backgammon_model.pth")


Epoch 010:
  Train Loss: 0.4733, Accuracy: 77.68%
  Val   Loss: 0.4584, Accuracy: 78.86%
  Test  Loss: 0.6295, Accuracy: 68.14%

Epoch 020:
  Train Loss: 0.4009, Accuracy: 82.06%
  Val   Loss: 0.3902, Accuracy: 82.85%
  Test  Loss: 0.6979, Accuracy: 68.61%

Epoch 030:
  Train Loss: 0.3616, Accuracy: 84.52%
  Val   Loss: 0.3491, Accuracy: 85.17%
  Test  Loss: 0.7538, Accuracy: 67.82%

Epoch 040:
  Train Loss: 0.3362, Accuracy: 85.63%
  Val   Loss: 0.3241, Accuracy: 86.52%
  Test  Loss: 0.8299, Accuracy: 67.43%

Epoch 050:
  Train Loss: 0.3153, Accuracy: 86.95%
  Val   Loss: 0.3021, Accuracy: 87.70%
  Test  Loss: 0.8996, Accuracy: 66.25%

Epoch 060:
  Train Loss: 0.2979, Accuracy: 87.75%
  Val   Loss: 0.2855, Accuracy: 88.75%
  Test  Loss: 0.9482, Accuracy: 65.93%

Epoch 070:
  Train Loss: 0.2828, Accuracy: 88.62%
  Val   Loss: 0.2708, Accuracy: 89.34%
  Test  Loss: 0.9899, Accuracy: 65.69%

Epoch 080:
  Train Loss: 0.2706, Accuracy: 89.18%
  Val   Loss: 0.2598, Accuracy: 89.85%
  Test  

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# Load data
def load_dataset(path):
    data = np.loadtxt(path, delimiter=",")
    X = torch.tensor(data[:, :-1], dtype=torch.float32)
    y = torch.tensor(data[:, -1], dtype=torch.float32).unsqueeze(1)
    return TensorDataset(X, y)

train_data = load_dataset(os.path.join("..","Data","Deep","Cube","Accept","train.txt"))
test_data = load_dataset(os.path.join("..","Data","Deep","Cube","Accept","test.txt"))

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64)

# Model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(289, 12)
        self.fc2 = nn.Linear(12, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(1, 101):
    model.train()
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if epoch % 10 == 0:
        model.eval()
        correct = total = 0
        with torch.no_grad():
            for X_test, y_test in test_loader:
                X_test, y_test = X_test.to(device), y_test.to(device)
                outputs = model(X_test)
                predictions = (outputs > 0.5).float()
                correct += (predictions == y_test).sum().item()
                total += y_test.size(0)
        accuracy = correct / total
        print(f"Epoch {epoch}, Test Accuracy: {accuracy:.4f}")
torch.save(model.state_dict(), "cube_accept_model.pth")

AttributeError: module 'torch' has no attribute 'ReLU'